In [24]:
# @title 1. Install Dependencies
# We force upgrade to ensure compatibility between LangChain core and integrations
!pip install -qU \
    langchain \
    langchain-community \
    langchain-chroma \
    langchain-huggingface \
    langchain-google-genai \
    chromadb \
    beautifulsoup4 \
    tiktoken

print("✅ Dependencies installed. Restart the session if prompted!")

✅ Dependencies installed. Restart the session if prompted!


In [25]:
# @title 2. Setup Environment & Keys
import os
from google.colab import userdata

# Securely fetch the key
try:
    GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')
    os.environ["GEMINI_API_KEY"] = GEMINI_API_KEY
    print("✅ API Key loaded securely.")
except:
    # Fallback if you haven't set it in the sidebar yet
    os.environ["GEMINI_API_KEY"] = input("Enter your Gemini API Key: ")

# Suppress warnings for a clean output
import warnings
warnings.filterwarnings("ignore")

✅ API Key loaded securely.


In [26]:
from langchain_huggingface import HuggingFaceEmbeddings # <--- Added this

# 1. Define the Embedding Model
# We use BAAI/bge-m3 as discussed (runs locally in Colab)
embeddings = HuggingFaceEmbeddings(
    model_name="BAAI/bge-m3",
    model_kwargs={'device': 'cuda'}, # Uses GPU if available
    encode_kwargs={'normalize_embeddings': True}
)

In [27]:
# @title 3. Ingest, Split, and Index Data
import bs4
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma

# --- CONFIGURATION ---
URLS = [
    "https://lilianweng.github.io/posts/2023-06-23-agent/",
    "https://lilianweng.github.io/posts/2023-03-15-prompt-engineering/"
]
EMBEDDING_MODEL = "BAAI/bge-m3"
# ---------------------

print("🔄 Loading content from URLs...")
# Only parse the article content (ignore headers/footers) for cleaner data
bs4_strainer = bs4.SoupStrainer(class_=("post-content", "post-title", "post-header"))
loader = WebBaseLoader(
    web_paths=URLS,
    bs_kwargs={"parse_only": bs4_strainer},
)
docs = loader.load()

print(f"   Loaded {len(docs)} documents.")

print("✂️ Splitting documents...")
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200, # Vital: keeps context between breaks
    add_start_index=True
)
splits = text_splitter.split_documents(docs)
print(f"   Created {len(splits)} chunks.")

print("🧠 Initializing Embeddings (this downloads the model)...")
embeddings = HuggingFaceEmbeddings(
    model_name=EMBEDDING_MODEL,
    model_kwargs={'device': 'cuda'}, # Use T4 GPU in Colab
    encode_kwargs={'normalize_embeddings': True} # Crucial for Cosine Similarity
)

print("💾 Indexing into Chroma Vector Store...")
vectorstore = Chroma.from_documents(
    documents=splits,
    embedding=embeddings,
    collection_name="expert_rag_db"
)
retriever = vectorstore.as_retriever(search_kwargs={"k": 5}) # Retrieve top 5 chunks

print("✅ Indexing Complete! The RAG base is ready.")

🔄 Loading content from URLs...
   Loaded 2 documents.
✂️ Splitting documents...
   Created 106 chunks.
🧠 Initializing Embeddings (this downloads the model)...
💾 Indexing into Chroma Vector Store...
✅ Indexing Complete! The RAG base is ready.


In [28]:
# @title 4. Initialize LLM & Create RAG Chain (Fixed)
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_google_genai import ChatGoogleGenerativeAI
from operator import itemgetter # <--- Import this

# 1. Setup the LLM
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash", # Updated to a stable model name
    temperature=0
)

# 2. Create the Prompt Template
# Note: I changed {input} to {question} to match your chain keys
system_prompt = (
    "You are an expert AI technical assistant. "
    "Use the following pieces of retrieved context to answer the user's question. "
    "If you don't know the answer, say that you don't know. "
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{question}"),
    ]
)

# 3. RAG Chain (Fixed Logic)
rag_chain = (
    {
        # explicitly get the "question" string for the retriever
        "context": itemgetter("question") | retriever,
        # explicitly pass the "question" string to the prompt
        "question": itemgetter("question")
    }
    | prompt
    | llm
    | StrOutputParser()
)

print("✅ RAG Agent is Online and Listening.")

✅ RAG Agent is Online and Listening.


In [29]:
# @title 5. Chat with your Data
question = "What is prompt engineering ?"

print(f"❓ Question: {question}\n")

# Invoke correctly
response = rag_chain.invoke({"question": question})

print("🤖 Answer:")
print(response)

❓ Question: What is prompt engineering ?

🤖 Answer:
Prompt Engineering, also known as In-Context Prompting, refers to methods used to communicate with a Large Language Model (LLM) to guide its behavior towards desired outcomes without modifying the model's underlying weights.

It is an empirical science that requires significant experimentation and heuristics, as the effectiveness of prompt engineering methods can vary greatly across different models. At its core, the goal of prompt engineering is to achieve alignment and steerability of the model. This field primarily focuses on autoregressive language models.
